In [1]:
import shutil
import os


original_dataset_dir = "./train"
train_cats_dir = "./train/cats"
train_dogs_dir = "./train/dogs"

fnames = ["dog.{}.jpg".format(i) for i in range(1000)]

for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = train_dogs_dir + "/" + fname
    shutil.copyfile(from_data, to_data)

fnames = ["cat.{}.jpg".format(i) for i in range(1000)]
 
for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = train_cats_dir + "/" + fname
    shutil.copyfile(from_data, to_data)
    
fnames = ["dog.{}.jpg".format(i) for i in range(1000)]
 
for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = train_dogs_dir + "/" + fname
    shutil.copyfile(from_data, to_data)

In [2]:
validation_cats_dir = "validation/cats"
validation_dogs_dir = "validation/dogs"


fnames = ["cat.{}.jpg".format(i) for i in range(1000,1500)]
 
for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = validation_cats_dir + "/" + fname
    shutil.copyfile(from_data, to_data)
 
fnames = ["dog.{}.jpg".format(i) for i in range(1000,1500)]
 
for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = validation_dogs_dir + "/" + fname
    shutil.copyfile(from_data, to_data)

In [3]:
test_cats_dir = "test/cats"
test_dogs_dir = "test/dogs"

fnames = ["cat.{}.jpg".format(i) for i in range(1500,2000)]
 
for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = test_cats_dir + "/" + fname
    shutil.copyfile(from_data, to_data)
 
fnames = ["dog.{}.jpg".format(i) for i in range(1500,2000)]
 
for fname in fnames:
    from_data = original_dataset_dir + "/" + fname
    to_data = test_dogs_dir + "/" + fname
    shutil.copyfile(from_data, to_data)

In [4]:
print("train cat:{}".format(len(os.listdir(train_cats_dir))))
print("train dog:{}".format(len(os.listdir(train_dogs_dir))))
 
print("validation cat:{}".format(len(os.listdir(validation_cats_dir))))
print("validation dog:{}".format(len(os.listdir(validation_dogs_dir))))
 
print("test cat:{}".format(len(os.listdir(test_cats_dir))))
print("test dog:{}".format(len(os.listdir(test_dogs_dir))))

train cat:1001
train dog:1001
validation cat:501
validation dog:501
test cat:501
test dog:501


In [ ]:
# ネットワークの作成
from keras import layers
from keras import models
 
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
 
model.add(layers.Flatten())
 
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))
 
model.summary()

In [ ]:
from keras import optimizers

# 2値分類では、活性化関数は「sigmoid」、損失関数は「binary_crossentropy」を使うのが一般的
model.compile(loss="binary_crossentropy",
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])

学習にかけるためには、画像ファイルに以下の前処理を行う

1. 画像ファイルを読み込む
1. 浮動小数点数型(float型)にする
1. ピクセル値(0-255)を、[0,1]の範囲の値にする

* target_sizeは、画像のサイズです。上では150,150のサイズにリサイズします。
* batch_sizeは、一度に処理する画像の枚数です。20枚を1バッチとします。
* class_modeは、”binary”として二値のラベルを作成します。

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import scipy

train_dir = "train"
validation_dir = "validation"

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode="binary"
)
 
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode="binary"
)

In [ ]:
for data,label in train_generator:
    print(data.shape)
    print(label.shape)
    break


(20, 150, 150, 3)
(20,)

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=30,
                             validation_data=validation_generator,
                             validation_steps=50)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
 

acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
 
epochs = range(1,len(acc) + 1)
 
plt.plot(epochs, acc,"bo",label="Training Acc")
plt.plot(epochs, val_acc,"b",label="Validation Acc")
plt.legend()
 
plt.figure()
 
plt.plot(epochs,loss,"bo",label="Training Loss")
plt.plot(epochs,val_loss,"b",label="Validation Loss")
plt.legend()
 
plt.show()